We now have a high-level understanding of how secure MPC works. In the following tutorials, we will describe how to use CrypTen to carry out secure operations on encrypted tensors. 

In this tutorial, we will introduce a fundamental building block in CrypTen, called ```CrypTensor```. Think of an  ```CrypTensor``` as an encrypted ```torch``` tensor that it can be used for computing securely on data. CrypTen currently only supports secure MPC protocols, so a ```CrypTensor``` is actually a tensor encrypted using secure MPC protocols, called ```MPCTensor```. We'll go into greater detail about ```MPCTensors``` in Tutorial 2. 

Let's begin by importing ```crypten``` and ```torch``` libraries. (If the imports fail, please see the installation instructions at <b>TODO</b>).

In [1]:
import torch
import crypten

crypten.init()

### Creating an Encrypted Tensor
CrypTen provides a ```crypten.cryptensor``` factory function, similar to ```torch.tensor```, to make creating ```CrypTensors``` easy. 

Let's create a example ```torch``` tensor, and use it to create a ```CrypTensor```. We can then make sure that the encryption worked correctly, and we get back our original tensor. Alternately, we can also create a ```CrypTensor``` directly from a list or an array, and decrypt it to get back the plaintext data in a ```torch``` tensor. 

In [2]:
#Create torch tensor
x = torch.tensor([1.0, 2.0, 3.0])

#Create the encrypted version of the tensor
x_enc = crypten.cryptensor(x)

#Decrypt the tensor to get the plain-text
x_dec = x_enc.get_plain_text()
print(x_dec)

tensor([1., 2., 3.])


In [3]:
#Create CrypTensor directly from data
x_enc = crypten.cryptensor([1.0, 2.0, 3.0])

#Decrypt the tensor to get the data in plain-text
x_dec = x_enc.get_plain_text()
print(x_dec)

tensor([1., 2., 3.])


You'll notice that we've used a function called ```get_plain_text``` to decrypt our ```CrypTensors```. We'll go into more detail on when and how this function can be used in Tutorial 3.

### Operations on Encrypted Tensors
Now let's look at what we can do with our ```CrypTensors```. We can carry out regular arithmetic operations between ```CrypTensors```, as well as between ```CrypTensors``` and plaintext tensors. 

In [4]:
#Arithmetic operations between CrypTensors and plaintext tensors
y_enc = x_enc + 2
print("addition:", y_enc.get_plain_text())

y_enc = x_enc - 2
print("subtraction:", y_enc.get_plain_text())

y_enc = x_enc * 2
print("multiplication:", y_enc.get_plain_text())

y_enc = x_enc / 2
print("division:", y_enc.get_plain_text())

addition: tensor([3., 4., 5.])
subtraction: tensor([-1.,  0.,  1.])
multiplication: tensor([2., 4., 6.])
division: tensor([0.5000, 1.0000, 1.5000])


In [5]:
#Arithmetic operations between two CrypTensors
y_enc = crypten.cryptensor([2.0])

z_enc = x_enc + y_enc
print("addition:", z_enc.get_plain_text())

z_enc = x_enc - y_enc
print("subtraction:", z_enc.get_plain_text())

z_enc = x_enc * y_enc
print("multiplication:", z_enc.get_plain_text())

z_enc = x_enc / y_enc
print("division:", z_enc.get_plain_text())

addition: tensor([3., 4., 5.])
subtraction: tensor([-1.,  0.,  1.])
multiplication: tensor([2., 4., 6.])
division: tensor([0.5000, 1.0000, 1.5000])


Like PyTorch, CrypTen supports both in-place and out-of-place operations.

Similarly, we can carry out binary comparisons with ```CrypTensors```. The following examples demonstrate how we do so. A comparison between a ```CrypTensor``` and a plaintext tensor always creates a  ```CrypTensor```. We can then decrypt this ```CrypTensor``` to examine whether the output of the binary comparison is 0 or 1.

In [6]:
#Construct two example CrypTensors
x = torch.tensor([2.0])
x_enc = crypten.cryptensor(x)
y = torch.tensor([4.0])
y_enc = crypten.cryptensor(y)

#Comparison between CrypTensor/plaintext tensor
comp1 = x_enc < y
print("comparison output encrypted check:", crypten.is_encrypted_tensor(comp1))
print("comparison 2 < 4:", comp1.get_plain_text())
print()
 
#Comparison between CrypTensor/CrypTensor
comp2 = (x_enc < y_enc)
print("comparison output encrypted check:", crypten.is_encrypted_tensor(comp2))
print("comparison 2 < 4:", comp2.get_plain_text())
print()

#Comparison between CrypTensor/CrypTensor
comp3 = (x_enc > y_enc)
print("comparison output encrypted check:", crypten.is_encrypted_tensor(comp3))
print("comparison 2 > 4:", comp3.get_plain_text())

comparison output encrypted check: True
comparison 2 < 4: tensor([1.])

comparison output encrypted check: True
comparison 2 < 4: tensor([1.])

comparison output encrypted check: True
comparison 2 > 4: tensor([0.])


Note that ```CrypTensors``` cannot be used directly in conditional expressions! Because the tensor is encrypted, the boolean expression cannot be evaluated. The ```CrypTensor``` always needs to be decrypted before evaluation in control-flow logic.

The following example illustrates how we can rewrite control-flow logic for ```CrypTensors```.

In [7]:
a, b = 2, 3

#unencrypted
if x < y:
    z = a
else:
    z = b

#encrypted
use_a = (x_enc < y_enc)
z_enc = use_a * a + (1 - use_a) * b
z = z_enc.get_plain_text()
print("The plaintext value of z:", z)

The plaintext value of z: tensor([2.])


One item to note in using ```CrypTensors```: For all arithmetic/binary operations involving ```CrypTensors```, the first operand <b>must</b> be a ```CrypTensor```. That is, for a ```CrypTensor```, ```x_enc``` and a plaintext tensor ```y```:
<ul>
<li>the expression ```x_enc < y``` is valid, but the mathematically equivalent expression ```y > x_enc``` will result in an error in CrypTen.</li>
<li>the statement ```x_enc + y``` is a valid operation in CrypTen, but the equivalent statement ```y + x_enc``` will result in an error.</li>
</ul>

CrypTen supports many of the operations that work on ```torch``` tensors. For example, we can stack ```CrypTensors```, reshape them, or get their sizes.

In [8]:
x_enc = crypten.cryptensor([1.0, 2.0, 3.0])
y_enc = crypten.cryptensor([4.0, 5.0, 6.0])

z_enc = crypten.stack([x_enc, y_enc])
print('Stack operation:')
print('Result tensor encrypted: ', crypten.is_encrypted_tensor(z_enc))
print('Result tensor plaintext:\n', z_enc.get_plain_text())
print()

w_enc = z_enc.reshape([-1,6])
print('Reshape operation:')
print('Result tensor encrypted: ', crypten.is_encrypted_tensor(w_enc))
print('Result tensor plaintext:\n', w_enc.get_plain_text())
print()

print('Sizes of CrypTensors:')
print('z_enc', z_enc.size())
print('w_enc', w_enc.size())

Stack operation:
Result tensor encrypted:  True
Result tensor plaintext:
 tensor([[1., 2., 3.],
        [4., 5., 6.]])

Reshape operation:
Result tensor encrypted:  True
Result tensor plaintext:
 tensor([[1., 2., 3., 4., 5., 6.]])

Sizes of CrypTensors:
z_enc torch.Size([2, 3])
w_enc torch.Size([1, 6])


For a complete list of all supported operations, please refer to the CrypTen documentation.